# LLM-Powered Recommendation Engine - Simple Demo

This notebook demonstrates how to use the LLM-powered recommendation engine for equine microbiome reports.

## Quick Start Guide

1. **Setup**: Copy `.env.example` to `.env` and add your API keys
2. **Install**: Run `poetry install --with llm` to install LLM dependencies
3. **Enable**: Set `ENABLE_LLM_RECOMMENDATIONS=true` in your `.env` file
4. **Run**: Execute the cells below to generate personalized recommendations

## 1. Setup and Import

In [ ]:
# Add the project root to Python path
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('.'))))

# Import the recommendation engine
from src.llm_recommendation_engine import create_recommendation_engine
from src.data_models import PatientInfo, MicrobiomeData
from src.csv_processor import CSVProcessor

print("✅ Imports successful!")

## 2. Check Configuration

In [ ]:
# Check if environment is configured
import os
from dotenv import load_dotenv

load_dotenv()

llm_enabled = os.getenv('ENABLE_LLM_RECOMMENDATIONS', 'false').lower() == 'true'
llm_provider = os.getenv('LLM_PROVIDER', 'not set')
has_api_key = bool(os.getenv(f'{llm_provider.upper()}_API_KEY', ''))

print(f"LLM Enabled: {llm_enabled}")
print(f"LLM Provider: {llm_provider}")
print(f"API Key Configured: {has_api_key}")

if not llm_enabled:
    print("\n⚠️  LLM is disabled. Set ENABLE_LLM_RECOMMENDATIONS=true in .env to enable.")
elif not has_api_key:
    print(f"\n⚠️  No API key found for {llm_provider}. Add it to your .env file.")

## 3. Create Patient Information

In [ ]:
# Create patient information
patient = PatientInfo(
    name="Thunder",
    age="12 years",
    sample_number="001",
    performed_by="Dr. Smith",
    requested_by="Jane Doe (Owner)"
)

print(f"Patient: {patient.name}, {patient.age}")
print(f"Sample: {patient.sample_number}")

## 4. Process Microbiome Data

You can either:
- Load real data from a CSV file
- Use example data (shown below)

In [ ]:
# Option 1: Load from CSV (uncomment to use)
# processor = CSVProcessor()
# microbiome_data = processor.process_csv('../data/sample_1.csv', 'barcode1')

# Option 2: Use example data
microbiome_data = MicrobiomeData(
    species_list=[],  # Would be populated from CSV
    phylum_distribution={
        "Bacillota": 45.0,
        "Bacteroidota": 2.5,  # Low - will trigger deficiency protocol
        "Pseudomonadota": 28.0,
        "Actinomycetota": 6.5,
        "Others": 18.0
    },
    dysbiosis_index=38.5,
    total_species_count=142,
    dysbiosis_category="mild",
    clinical_interpretation="Mild dysbiosis detected",
    recommendations=[],
    parasite_results=[],
    microscopic_results=[],
    biochemical_results=[]
)

print("Microbiome Analysis Summary:")
print(f"  Dysbiosis Index: {microbiome_data.dysbiosis_index}")
print(f"  Category: {microbiome_data.dysbiosis_category}")
print(f"  Total Species: {microbiome_data.total_species_count}")
print("\nPhylum Distribution:")
for phylum, percentage in microbiome_data.phylum_distribution.items():
    print(f"  {phylum}: {percentage:.1f}%")

## 5. Generate LLM Recommendations

In [ ]:
# Create the recommendation engine
engine = create_recommendation_engine()

# Optional: Add clinical history
clinical_history = {
    "recent_antibiotics": False,
    "chronic_issues": False,
    "symptoms": "Occasional loose stools, dull coat"
}

# Generate recommendations
print("🤖 Generating recommendations...")
results = engine.process_sample(
    microbiome_data=microbiome_data,
    patient_info=patient,
    clinical_history=clinical_history
)

print("✅ Complete!")

## 6. View Results

In [ ]:
# Display selected template
print("=" * 60)
print("SELECTED CLINICAL TEMPLATE")
print("=" * 60)
print(f"Scenario: {results['template_info']['scenario']}")
print(f"Title: {results['template_info']['title']}")
print(f"Confidence: {results['template_info']['confidence'] * 100:.0f}%")
print()

In [ ]:
# Display LLM recommendations
recs = results['recommendations']

print("=" * 60)
print("PERSONALIZED RECOMMENDATIONS")
print("=" * 60)

if recs.get('llm_generated', False):
    print("✨ Generated by LLM" + (" (cached)" if recs.get('from_cache', False) else ""))
else:
    print("📋 Using template-based recommendations")

print(f"\nClinical Interpretation:")
print(f"{recs['clinical_interpretation']}")

print(f"\nDietary Modifications:")
for i, mod in enumerate(recs['dietary_modifications'], 1):
    print(f"  {i}. {mod}")

print(f"\nSupplement Protocol:")
for i, supp in enumerate(recs['supplement_protocol'], 1):
    print(f"  {i}. {supp}")

print(f"\nManagement Changes:")
for i, change in enumerate(recs['management_changes'], 1):
    print(f"  {i}. {change}")

print(f"\nMonitoring: {recs['monitoring_plan']}")
print(f"Follow-up: {recs['follow_up']}")

## 7. Try Different Scenarios

Experiment with different microbiome profiles to see how recommendations change:

In [ ]:
# Example: Severe dysbiosis case
severe_case = MicrobiomeData(
    species_list=[],
    phylum_distribution={
        "Bacillota": 82.0,  # Very high!
        "Bacteroidota": 3.0,
        "Pseudomonadota": 12.0,
        "Actinomycetota": 1.0,
        "Others": 2.0
    },
    dysbiosis_index=78.5,
    total_species_count=45,  # Low diversity
    dysbiosis_category="severe",
    clinical_interpretation="Severe dysbiosis",
    recommendations=[],
    parasite_results=[],
    microscopic_results=[],
    biochemical_results=[]
)

# Generate recommendations for severe case
severe_results = engine.process_sample(
    microbiome_data=severe_case,
    patient_info=patient
)

print(f"Severe Case Template: {severe_results['template_info']['title']}")
print(f"\nFirst Recommendation: {severe_results['recommendations']['dietary_modifications'][0]}")

## 8. Integration with Report Generation

Here's how to integrate LLM recommendations into your PDF reports:

In [ ]:
from src.report_generator import ReportGenerator

# The report generator can now use LLM recommendations
# when ENABLE_LLM_RECOMMENDATIONS=true

# Example integration (requires actual CSV file)
# generator = ReportGenerator(language='en')
# success = generator.generate_report(
#     csv_path='../data/sample_1.csv',
#     patient_info=patient,
#     output_path='../reports/report_with_llm.pdf'
# )

print("To generate reports with LLM recommendations:")
print("1. Set ENABLE_LLM_RECOMMENDATIONS=true in .env")
print("2. Configure your preferred LLM provider and API key")
print("3. Run the report generator as usual")
print("\nThe LLM will automatically enhance recommendations based on the microbiome data!")

## Summary

This notebook demonstrated:

1. **Easy Setup**: Simple configuration via `.env` file
2. **Smart Template Selection**: Automatically selects from 8 clinical scenarios
3. **LLM Enhancement**: Personalizes recommendations using AI
4. **Fallback Support**: Works even without LLM (uses templates)
5. **Multiple Providers**: Supports OpenAI, Anthropic, and Google Gemini

### Next Steps

- Configure your API keys in `.env`
- Try different microbiome profiles
- Generate full PDF reports with LLM recommendations
- Monitor recommendation quality and adjust prompts as needed